In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

interval = 5
#dataset = pd.read_csv(os.path.join('interval_datasets', str(interval) + '.csv'), index_col=0)

dataset = pd.read_csv('interval_datasets/AS14.09_5.csv')

In [24]:
X = np.asarray(dataset[['mood', 'arousal', 'valence', 'activity']])
y = dataset['predicted_mood']

In [25]:
np.random.seed(43)
data = zip(X,y)
np.random.shuffle(data)

X, y = zip(*data)

In [26]:
datasize = len(X)

X_test, y_test = X[:datasize/5], y[:datasize/5]
X_train, y_train = X[datasize/5:], y[datasize/5:]

In [38]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

tol_list = [1e-4, 1e-3, 1e-2, 1e-1]
C_range = [0.1, 0.5, 1, 1.5]
eps_range = [0.01, 0.1, 0.5]
kernel_list = ['linear', 'rbf', 'sigmoid', 'poly']

param_grid = dict(C=C_range, epsilon=eps_range, kernel=kernel_list, tol=tol_list)
grid = GridSearchCV(svm.SVR(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'C': [0.1, 0.5, 1, 1.5], 'tol': [0.0001, 0.001, 0.01, 0.1], 'epsilon': [0.01, 0.1, 0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [49]:
print grid.best_score_, grid.best_params_
#print grid.
best_score_index = pd.DataFrame(grid.cv_results_)['mean_test_score'].argmax()
pd.DataFrame(grid.cv_results_).loc[175]

-0.29867931429605116 {'epsilon': 0.1, 'C': 1.5, 'tol': 0.1, 'kernel': 'poly'}


mean_fit_time                                               0.000959063
mean_score_time                                             0.000257111
mean_test_score                                               -0.298679
mean_train_score                                              -0.263936
param_C                                                             1.5
param_epsilon                                                       0.1
param_kernel                                                       poly
param_tol                                                           0.1
params                {u'epsilon': 0.1, u'C': 1.5, u'tol': 0.1, u'ke...
rank_test_score                                                       1
split0_test_score                                             -0.808491
split0_train_score                                            -0.159737
split1_test_score                                              -0.35936
split1_train_score                                            -0

In [34]:
clf = svm.SVR(C=1.5, epsilon=0.1, kernel='poly', tol=0.1)
clf.fit(X_train, y_train) 
predicted = clf.predict(X_test)
y_test

mse = ((predicted - y_test) ** 2).mean(axis=None)

In [35]:
mse

0.4499066958797757

In [25]:
plt.plot(range(len(y_test)), y_test, range(len(y_test)), predicted)
plt.show()